# Loop 1 LB Feedback Analysis

## Submission Results
- CV Score: 0.8283
- LB Score: 0.7727
- Gap: +0.0556 (CV is optimistic by ~5.6%)

## Key Questions
1. Why is CV so much higher than LB?
2. Is there distribution shift between train and test?
3. What can we learn from the prediction patterns?

In [1]:
import pandas as pd
import numpy as np

# Load data
train = pd.read_csv('/home/data/train.csv')
test = pd.read_csv('/home/data/test.csv')
submission = pd.read_csv('/home/code/submission_candidates/candidate_000.csv')

print('Train shape:', train.shape)
print('Test shape:', test.shape)
print('Submission shape:', submission.shape)
print('\nSubmission prediction distribution:')
print(submission['Survived'].value_counts(normalize=True))

Train shape: (891, 12)
Test shape: (418, 11)
Submission shape: (418, 2)

Submission prediction distribution:
Survived
0    0.624402
1    0.375598
Name: proportion, dtype: float64


In [2]:
# Compare distributions between train and test
print('=== Distribution Comparison ===')
print('\n--- Pclass ---')
print('Train:', train['Pclass'].value_counts(normalize=True).sort_index().values)
print('Test:', test['Pclass'].value_counts(normalize=True).sort_index().values)

print('\n--- Sex ---')
print('Train:', train['Sex'].value_counts(normalize=True))
print('Test:', test['Sex'].value_counts(normalize=True))

print('\n--- Age (mean, std) ---')
print(f'Train: {train["Age"].mean():.2f}, {train["Age"].std():.2f}')
print(f'Test: {test["Age"].mean():.2f}, {test["Age"].std():.2f}')

print('\n--- Fare (mean, std) ---')
print(f'Train: {train["Fare"].mean():.2f}, {train["Fare"].std():.2f}')
print(f'Test: {test["Fare"].mean():.2f}, {test["Fare"].std():.2f}')

=== Distribution Comparison ===

--- Pclass ---
Train: [0.24242424 0.20650954 0.55106622]
Test: [0.25598086 0.22248804 0.5215311 ]

--- Sex ---
Train: Sex
male      0.647587
female    0.352413
Name: proportion, dtype: float64
Test: Sex
male      0.636364
female    0.363636
Name: proportion, dtype: float64

--- Age (mean, std) ---
Train: 29.70, 14.53
Test: 30.27, 14.18

--- Fare (mean, std) ---
Train: 32.20, 49.69
Test: 35.63, 55.91


In [3]:
# Check embarked distribution
print('--- Embarked ---')
print('Train:', train['Embarked'].value_counts(normalize=True))
print('Test:', test['Embarked'].value_counts(normalize=True))

# Check SibSp and Parch
print('\n--- SibSp ---')
print('Train:', train['SibSp'].value_counts(normalize=True).head())
print('Test:', test['SibSp'].value_counts(normalize=True).head())

print('\n--- Parch ---')
print('Train:', train['Parch'].value_counts(normalize=True).head())
print('Test:', test['Parch'].value_counts(normalize=True).head())

--- Embarked ---
Train: Embarked
S    0.724409
C    0.188976
Q    0.086614
Name: proportion, dtype: float64
Test: Embarked
S    0.645933
C    0.244019
Q    0.110048
Name: proportion, dtype: float64

--- SibSp ---
Train: SibSp
0    0.682379
1    0.234568
2    0.031425
4    0.020202
3    0.017957
Name: proportion, dtype: float64
Test: SibSp
0    0.677033
1    0.263158
2    0.033493
3    0.009569
4    0.009569
Name: proportion, dtype: float64

--- Parch ---
Train: Parch
0    0.760943
1    0.132435
2    0.089787
5    0.005612
3    0.005612
Name: proportion, dtype: float64
Test: Parch
0    0.775120
1    0.124402
2    0.078947
3    0.007177
4    0.004785
Name: proportion, dtype: float64


In [4]:
# Extract Title and compare
def extract_title(df):
    df = df.copy()
    df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    title_mapping = {
        'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs',
        'Lady': 'Rare', 'Countess': 'Rare', 'Capt': 'Rare',
        'Col': 'Rare', 'Don': 'Rare', 'Dr': 'Rare',
        'Major': 'Rare', 'Rev': 'Rare', 'Sir': 'Rare',
        'Jonkheer': 'Rare', 'Dona': 'Rare'
    }
    df['Title'] = df['Title'].replace(title_mapping)
    return df

train_t = extract_title(train)
test_t = extract_title(test)

print('--- Title Distribution ---')
print('Train:')
print(train_t['Title'].value_counts(normalize=True))
print('\nTest:')
print(test_t['Title'].value_counts(normalize=True))

--- Title Distribution ---
Train:
Title
Mr        0.580247
Miss      0.207632
Mrs       0.141414
Master    0.044893
Rare      0.025814
Name: proportion, dtype: float64

Test:
Title
Mr        0.574163
Miss      0.188995
Mrs       0.172249
Master    0.050239
Rare      0.014354
Name: proportion, dtype: float64


In [5]:
# Analyze survival rates by key features in training data
print('=== Survival Rates in Training Data ===')
print('\n--- By Sex ---')
print(train.groupby('Sex')['Survived'].mean())

print('\n--- By Pclass ---')
print(train.groupby('Pclass')['Survived'].mean())

print('\n--- By Title ---')
print(train_t.groupby('Title')['Survived'].mean())

=== Survival Rates in Training Data ===

--- By Sex ---
Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64

--- By Pclass ---
Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64

--- By Title ---
Title
Master    0.575000
Miss      0.702703
Mr        0.156673
Mrs       0.793651
Rare      0.347826
Name: Survived, dtype: float64


In [6]:
# Check if test has different Sex/Pclass combinations
print('=== Sex-Pclass Combinations ===')
train['Sex_Pclass'] = train['Sex'] + '_' + train['Pclass'].astype(str)
test['Sex_Pclass'] = test['Sex'] + '_' + test['Pclass'].astype(str)

print('Train:')
print(train['Sex_Pclass'].value_counts(normalize=True).sort_index())
print('\nTest:')
print(test['Sex_Pclass'].value_counts(normalize=True).sort_index())

=== Sex-Pclass Combinations ===
Train:
Sex_Pclass
female_1    0.105499
female_2    0.085297
female_3    0.161616
male_1      0.136925
male_2      0.121212
male_3      0.389450
Name: proportion, dtype: float64

Test:
Sex_Pclass
female_1    0.119617
female_2    0.071770
female_3    0.172249
male_1      0.136364
male_2      0.150718
male_3      0.349282
Name: proportion, dtype: float64


In [7]:
# Calculate expected survival rate based on training patterns
# If test has different distribution, expected rate would differ

# Get survival rates by Sex_Pclass from training
survival_rates = train.groupby('Sex_Pclass')['Survived'].mean()
print('Survival rates by Sex_Pclass (from training):')
print(survival_rates)

# Apply to test distribution
test_dist = test['Sex_Pclass'].value_counts(normalize=True)
expected_survival = sum(survival_rates[sp] * test_dist[sp] for sp in test_dist.index)
print(f'\nExpected survival rate in test (based on train patterns): {expected_survival:.4f}')
print(f'Actual train survival rate: {train["Survived"].mean():.4f}')
print(f'Our prediction survival rate: {submission["Survived"].mean():.4f}')

Survival rates by Sex_Pclass (from training):
Sex_Pclass
female_1    0.968085
female_2    0.921053
female_3    0.500000
male_1      0.368852
male_2      0.157407
male_3      0.135447
Name: Survived, dtype: float64

Expected survival rate in test (based on train patterns): 0.3894
Actual train survival rate: 0.3838
Our prediction survival rate: 0.3756


In [8]:
# Key insight: The CV-LB gap of 0.0556 is substantial
# Possible causes:
# 1. Overfitting to training data patterns
# 2. Distribution shift in test set
# 3. Small test set variance (418 samples)

# The LB score of 0.7727 means we got ~323 correct out of 418
# CV score of 0.8283 would mean ~738 correct out of 891 in training

print('=== Gap Analysis ===')
print(f'CV accuracy: 0.8283 -> ~{int(0.8283 * 891)} correct out of 891')
print(f'LB accuracy: 0.7727 -> ~{int(0.7727 * 418)} correct out of 418')
print(f'Gap: 5.56 percentage points')
print('\nThis gap suggests our model may be overfitting to training patterns')
print('or there is distribution shift between train and test.')

=== Gap Analysis ===
CV accuracy: 0.8283 -> ~738 correct out of 891
LB accuracy: 0.7727 -> ~322 correct out of 418
Gap: 5.56 percentage points

This gap suggests our model may be overfitting to training patterns
or there is distribution shift between train and test.
